In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://dlcdn.apache.org/spark/
!wget -q https://dlcdn.apache.org/spark/spark-3.4.4/spark-3.4.4-bin-hadoop3.tgz
!tar xf spark-3.4.4-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j
import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.4.4-bin-hadoop3"
import findspark
findspark.init()
findspark.find()
import pyspark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,607 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,863 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-upda

In [4]:
import pandas as pd
from pyspark.sql import functions as f

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .getOrCreate()
spark

In [10]:
from pyspark.sql import types as T

data_schema = T.StructType([
    T.StructField('contract_id', T.StringType(), False),
    T.StructField('timestamp', T.TimestampType(), False),
    T.StructField('value', T.DoubleType(), True),
    T.StructField('value_source', T.StringType(), False),
    T.StructField('annotations', T.StringType(), True)
])

df_raw_time_series = spark.read.format('json') \
    .schema(data_schema) \
    .load('/content/drive/MyDrive/Colab Notebooks/project/raw_time_series/json')
df_raw_time_series = spark.createDataFrame(df_raw_time_series.rdd, schema=data_schema)
df_raw_time_series.printSchema()
df_raw_time_series.show()

root
 |-- contract_id: string (nullable = false)
 |-- timestamp: timestamp (nullable = false)
 |-- value: double (nullable = true)
 |-- value_source: string (nullable = false)
 |-- annotations: string (nullable = true)

+-------------------+-------------------+--------------------+------------+--------------------+
|        contract_id|          timestamp|               value|value_source|         annotations|
+-------------------+-------------------+--------------------+------------+--------------------+
| 04_02_111 _ CHR12 |2023-01-01 06:00:00| 0.02591860654732236| measurement|{"region":"Europe...|
| 04 _02_111 _CHR12 |2023-01-01 17:00:00| 0.07385444264936832| measurement|{"region":"Europe...|
| 04_02_111 _ CHR12 |2023-01-01 17:30:22| 0.08180149515221906| measurement|{"region":"Europe...|
| 04 _02_111 _CHR12 |2023-01-01 21:30:00| 0.08670661371854547| measurement|{"region":"Europe...|
|04 _ 02 _111_CHR12 |2023-01-02 00:30:00| 0.03597601881331959| measurement|{"region":"Europe...|
|04 

In [11]:
data_schema = T.StructType([
    T.StructField('contract_id', T.StringType(), False),
    T.StructField('target_local_start_timestamp', T.TimestampType(), False),
    T.StructField('target_local_end_timestamp', T.TimestampType(), False),
    T.StructField('tariff_name', T.StringType(), True),
    T.StructField('charge_type', T.StringType(), False),
    T.StructField('price', T.DoubleType(), True)
])

df_customer_tariff = spark.read.format('json') \
    .schema(data_schema) \
    .load('/content/drive/MyDrive/Colab Notebooks/project/customer_tariff/json')
df_customer_tariff = spark.createDataFrame(df_customer_tariff.rdd, schema=data_schema)
df_customer_tariff.printSchema()
df_customer_tariff.show()

root
 |-- contract_id: string (nullable = false)
 |-- target_local_start_timestamp: timestamp (nullable = false)
 |-- target_local_end_timestamp: timestamp (nullable = false)
 |-- tariff_name: string (nullable = true)
 |-- charge_type: string (nullable = false)
 |-- price: double (nullable = true)

+---------------+----------------------------+--------------------------+-----------------+-----------+------+
|    contract_id|target_local_start_timestamp|target_local_end_timestamp|      tariff_name|charge_type| price|
+---------------+----------------------------+--------------------------+-----------------+-----------+------+
|04_02_111_CHR28|         2022-12-01 00:00:00|       2023-02-16 00:00:00|     Electric Pro|        buy| 0.302|
|04_02_111_CHR28|         2023-02-16 00:00:00|       2024-05-19 00:00:00| Electric Loyalty|        buy|0.3938|
|04_02_111_CHR28|         2024-05-19 00:00:00|       2024-08-31 00:00:00|     Eco Electric|        buy|0.2095|
|04_02_111_CHR28|         2024-08-

Data clean-up and info extraction progress - if ruled more than once, all values will be turned to NULL

In [12]:
region_schema = T.StructType([
    T.StructField('region', T.StringType(), False)
])

events_schema = T.StructType([
    T.StructField("events", T.StructType([
        T.StructField("EV", T.StringType()),
        T.StructField("PV", T.StringType()),
        T.StructField("BATTERY_IN", T.StringType()),
        T.StructField("BATTERY_OUT", T.StringType()),
        T.StructField("GRID_SELL", T.StringType()),
    ]))
])

df_raw_time_series = (
    df_raw_time_series
    .withColumn('contract_id',
                f.regexp_replace(f.col('contract_id'), ' ', ''))
    .withColumn('value_source',
                f.when(f.col('value').isNull(), 'missing').otherwise(f.col('value_source')))
    .withColumn('timestamp',
                f.from_unixtime(f.round(f.unix_timestamp('timestamp') / (15 * 60)) * (15 * 60)))
    .withColumn('timestamp',
                f.date_format('timestamp', 'yyyy-MM-dd HH:mm'))
    .withColumn('region',
                f.from_json(f.col('annotations'), region_schema).getField('region'))
    .filter(~((f.col('region').isNull()) |
            (f.trim(f.col('region')) == '') |
            (f.regexp_extract(f.col('region'), r'^[A-Za-z]+(?:[\s-][A-Za-z]+)*\/[A-Za-z]+(?:[\s-][A-Za-z]+)*$', 0) == '')))
    .withColumn('utc_date',
                f.date_format('timestamp', 'yyyy-MM-dd'))
    .withColumn("local_timestamp",
                f.from_utc_timestamp(f.col('timestamp'), f.col("region")))
    .withColumn("local_date",
                f.to_date(f.col("local_timestamp")))
    .withColumn("annotation_json",
                f.from_json(f.col("annotations"), events_schema))
    .withColumn("sent_to_ev",
                f.coalesce(f.col("annotation_json.events.EV").cast(T.DoubleType()), f.lit(0)))
    .withColumn("received_from_pv",
                f.coalesce(f.col("annotation_json.events.PV").cast(T.DoubleType()), f.lit(0)))
    .withColumn("sent_to_battery",
                f.coalesce(f.col("annotation_json.events.BATTERY_IN").cast(T.DoubleType()), f.lit(0)))
    .withColumn("received_from_battery",
                f.coalesce(f.col("annotation_json.events.BATTERY_OUT").cast(T.DoubleType()), f.lit(0)))
    .withColumn("sent_to_grid",
                f.coalesce(f.col("annotation_json.events.GRID_SELL").cast(T.DoubleType()), f.lit(0)))
    .drop("annotation_json")
    .drop("annotations")
    .withColumn("received_from_grid",
                f.col("sent_to_ev") + f.col("sent_to_battery") + f.col("sent_to_grid") - f.col("received_from_pv") - f.col("received_from_battery"))
)

df_raw_time_series.show()

+---------------+----------------+--------------------+------------+-------------+----------+-------------------+----------+----------+----------------+---------------+---------------------+------------+------------------+
|    contract_id|       timestamp|               value|value_source|       region|  utc_date|    local_timestamp|local_date|sent_to_ev|received_from_pv|sent_to_battery|received_from_battery|sent_to_grid|received_from_grid|
+---------------+----------------+--------------------+------------+-------------+----------+-------------------+----------+----------+----------------+---------------+---------------------+------------+------------------+
|04_02_111_CHR12|2023-01-01 06:00| 0.02591860654732236| measurement|Europe/Berlin|2023-01-01|2023-01-01 07:00:00|2023-01-01|       0.0|             0.0|            0.0|                  0.0|         0.0|               0.0|
|04_02_111_CHR12|2023-01-01 17:00| 0.07385444264936832| measurement|Europe/Berlin|2023-01-01|2023-01-01 18:0

By cleaning the data and completing the missing values, we can now move on to the association with the consumers tariffs:

- the association of consumption entries with the tariffs, based on the contract number, time and type of price, purchase or sale.
- the pricing on purchase (`buy`) is the purchase price for consumer entry, multiplied by the `received_from_grid` column
- the pricing on sale (`sell`) is the sale price for consumption entry, multiplied by the `sent_to_grid` column.


In [16]:
df_joined = df_raw_time_series.join(df_customer_tariff, on=['contract_id'], how='left')

# filter the entries within the valid time range of the tariff
df_joined = df_joined.filter(f.col("timestamp") >= f.col("target_local_start_timestamp")) \
                    .filter(f.col("timestamp") <= f.col("target_local_end_timestamp"))

# calculate the priced value based on the charge type
df_joined = df_joined.withColumn("priced_value",
                                f.when(f.col("charge_type") == "buy", f.col("price") * f.col("received_from_grid")) \
                                .otherwise(f.when(f.col("charge_type") == "sell", f.col("price") * f.col("sent_to_grid"))) \
                               )

df_joined.orderBy(f.col('priced_value').desc()).show(500)


+---------------+----------------+------------------+------------+-------------+----------+-------------------+----------+----------+----------------+---------------+---------------------+------------+------------------+----------------------------+--------------------------+-----------------+-----------+------+------------+
|    contract_id|       timestamp|             value|value_source|       region|  utc_date|    local_timestamp|local_date|sent_to_ev|received_from_pv|sent_to_battery|received_from_battery|sent_to_grid|received_from_grid|target_local_start_timestamp|target_local_end_timestamp|      tariff_name|charge_type| price|priced_value|
+---------------+----------------+------------------+------------+-------------+----------+-------------------+----------+----------+----------------+---------------+---------------------+------------+------------------+----------------------------+--------------------------+-----------------+-----------+------+------------+
|04_02_111_CHR11|20

After association, we can move on to the calculation of the factors - per day / week / month / year, for each customer, to calculate:
- total energy consumption (the sum of the `value` column) - `kWh_total`
- the energy extracted from the battery and the PV - `kWh_From_battery`,` kWh_from_PY`
- the energy used for EV - `kWh_for_EV`
- the energy consumption used from the network - `kWh_from_grif`
- the cost of energy consumption used from the network - `price_billed`
- energy consumption sent to the network - `kWh_to_grid`
- the cost received back, of the energy consumption sent to the network - `price_cashback`
- total cost (cost difference) - `price_final`
- the aggregate calculation code is done during any period of time in a position, and for each interval, day / week / month / year, run this function. Here, it is displayed, for each period, the top 10 customers with the highest price.


In [ ]:
# tba
